## Lets take an example

In [ ]:
!pip install openai

In [ ]:
import openai
from getpass import getpass
openai.api_key = getpass()

## Lets try default model

In [ ]:
import openai

response = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": """Author David Chanoff has collaborated with a U.S. Navy admiral who served as the ambassador to the United Kingdom under which President?
         """}
    ],
    temperature=0
)

In [ ]:
print(response['choices'][0]['message']['content'])

## Lets try CoT

In [ ]:
import openai

response = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Author David Chanoff has collaborated with a U.S. Navy admiral who served as the ambassador to the United Kingdom under which President? Lets think step by step and give final answer"}
    ],
    temperature=0
)

In [ ]:
print(response['choices'][0]['message']['content'])

### <span style="color:#3366ff">Below is issues with CoT reasoning:</span>
- "chain-of-thought" reasoning is a static black box
- It uses its own internal representations to generate thoughts and is not grounded in the external world
- It has limited ability to reason reactively or update its knowledge

## Lets look into another example

In [ ]:
import openai

response = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Who won 2023 Indian Premier League?"}
    ]
)

In [ ]:
print(response['choices'][0]['message']['content'])

#### LLM is can not answer on recent events as its not connected to external world

# <span style="color:#3366ff">Problem Solving Process</span>

As a human, we often follow the <span style="color:#33cc33">thought-act-observation model</span> to solve problems. Let's consider the following simple example:

1. <span style="color:#9933ff">**Thought**</span>: I want to make a Sandwich, so I need to check if I have the recipe.
2. <span style="color:#ff9900">**Act**</span>: [<span style="color:#cc0000">Search</span>] for the recipe for Sandwich.
3. <span style="color:#ffcc00">**Observation**</span>: To prepare the Sandwich, I need onion and tomato. Therefore, I realize that I need to order onion and tomato online.
4. <span style="color:#9933ff">**Thought**</span>: I can order onion and tomato online.
5. <span style="color:#ff9900">**Act**</span>: [<span style="color:#cc0000">Search</span>] for onion and tomato ordering online.
6. <span style="color:#ffcc00">**Observation**</span>: I have successfully obtained onion and tomato through online ordering.
7. <span style="color:#9933ff">**Thought**</span>: Now, I have everything I need to prepare a Sandwich.
8. <span style="color:#ff9900">**Act**</span>: [<span style="color:#cc0000">Finish</span>] I can start preparing for the Sandwich.

By following this thought-act-observation model, I, as a human <span style="color:#ff0000">(agent)</span>, am able to solve the problem systematically, ensuring that I take the necessary actions and evaluate the outcomes at each step. An <span style="color:#ff0000">agent</span> refers to a person or entity capable of making decisions and taking actions in response to a given situation or problem. In this context, I am the <span style="color:#ff0000">agent</span> who is actively thinking, acting, and observing in order to solve the problem at hand.

Let's see how we can apply this problem-solving approach to our current problem.

In [ ]:
## we will use Langchain's implementation of ReAct to find answer to first problem
!pip install --upgrade openai
!pip install --upgrade langchain
!pip install wikipedia
!pip install google-search-results
!pip install --upgrade python-dotenv

In [ ]:
# import libraries
import openai
import os
from langchain.llms import OpenAI
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from dotenv import load_dotenv
load_dotenv()

# load API keys; you will need to obtain these if you haven't yet
os.environ["OPENAI_API_KEY"] = getpass()

In [ ]:
from langchain.utilities import WikipediaAPIWrapper

In [ ]:
from langchain import OpenAI, Wikipedia
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain.agents.react.base import DocstoreExplorer
docstore=DocstoreExplorer(Wikipedia())
tools = [
    Tool(
        name="Search",
        func=docstore.search,
        description="useful for when you need to ask with search"
    ),
    Tool(
        name="Lookup",
        func=docstore.lookup,
        description="useful for when you need to ask with lookup"
    )
]

llm = OpenAI(temperature=0, model_name="gpt-3.5-turbo")
react = initialize_agent(tools, llm, agent=AgentType.REACT_DOCSTORE, verbose=True)

In [ ]:
react.run("Author David Chanoff has collaborated with a U.S. Navy admiral who served as the ambassador to the United Kingdom under which President?")

In [ ]:
print(react.agent.llm_chain.prompt.template)

## Lets see if we can get the information from Internet about 2nd problem

In [ ]:
os.environ["SERPAPI_API_KEY"] = getpass("Add Serp API")

In [ ]:
llm = OpenAI(model_name="gpt-3.5-turbo" ,temperature=0)
tools = load_tools(["serpapi"], llm=llm)
agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)

In [ ]:
agent.run("Who won 2023 Indian Premier League?")

#### <span style="color:#3366ff">ReAct framework demonstrates its ability to auto-correct and adapt in real-time, improving the overall reliability and resilience of the system</span>

In [ ]:
## We can see the prompt used here
print(agent.agent.llm_chain.prompt.template)